In [4]:
data<-read.csv("Forces3.csv")

In [4]:
#Check for categorical data. The 3th and 96th columns are categorical
data[1:5,1:5]

predictor,predictor.1,predictor.2,predictor.3,predictor.4
3.0990676,30.31381,174.9512,8,301.3000
1.6074460,17.83223,212.2065,99,375.8420
4.8223026,39.69040,199.9276,15,151.1736
0.6624255,65.08108,175.8600,8,242.8898
1.5118722,78.65205,131.7224,9,250.1637


In [675]:
#Graphing the data, it looks like they are linear or quadratic, with large standard deviation
jpeg("all_data.jpg", width=900, height=30000, res=120)
par(mfrow=c(96,3),mar = rep(2, 4))
for (i in (1:3)){
    for (j in (1:96)){
        plot(data[,j],data[,96+i])
    }
}
dev.off()

png 
  2

In [28]:
#Shuffle rows before separation into x and y
data<- data[sample(nrow(data)),]

In [29]:
#Seperate
X_data=data[-c(97:99)]
y_data=data[c(97:99)]

In [30]:
#A little bit of stack overflow borrowing, the model.moatrix 
#line returns on hot encoded columns for things that are factors
ohe_cols<-function(x){
     xf=factor(x)
     return(model.matrix(~xf+0))
}

In [31]:
#One hot encodes specified columns of data, using above function
ohe_matrix<-function(X,cols){
    X<-cbind(X,lapply(X[cols], ohe_cols))
    X<-X[-cols]
    return(X)
}

In [32]:
#Apply oh encoder to the data 
cols=c(4,96)
X_data<-ohe_matrix(X_data, cols)

In [33]:
X_train<-X_data[1:(nrow(X_data)/100*70),]
y_train<-y_data[1:(nrow(y_data)/100*70),]

In [34]:
nrow(y_data)/100*85-nrow(X_data)/100*70

[1] 19401

In [35]:
X_val<-X_data[(nrow(X_data)/100*70):(nrow(X_data)/100*85),]
y_val<-y_data[(nrow(y_data)/100*70):(nrow(X_data)/100*85),]
X_test<-X_data[(nrow(X_data)/100*85):(nrow(X_data)),]
y_test<-y_data[(nrow(y_data)/100*85):(nrow(X_data)),]

In [36]:
#Clear some unused variables
rm(X_data)
rm(y_data)

In [37]:
#Calculate values for normalization
# X_tr_mean<-lapply(X_train,mean)
# X_tr_std<-lapply(X_train, sd)
# y_tr_mean<-lapply(y_train,mean)
# y_tr_std<-lapply(y_train, sd)

In [38]:
#Normalize X
# X_tr_norm<-(X_train-X_tr_mean)/X_tr_std
# X_val_norm<-(X_val-X_tr_mean)/X_tr_std
# X_test_norm<-(X_test-X_tr_mean)/X_tr_std

In [39]:
#Normalize y
# y_tr_norm<-(y_train-y_tr_mean)/y_tr_std
# y_val_norm<-(y_val-y_tr_mean)/y_tr_std
# y_test_norm<-(y_test-y_tr_mean)/y_tr_std

In [40]:
#Min max normalize
norm<-function(X,X_tr){
    for(i in 1:ncol(X)){
        X[,i]<-(X[,i] - min(X_tr[,i]))/(
        max(X_tr[,i]) - min(X_tr[,i]))
    }
    return(X)
}

In [41]:
X_tr_norm<-norm(X_train,X_train)
X_val_norm<-norm(X_val,X_train)
X_test_norm<-norm(X_test,X_train)

y_tr_norm<-norm(y_train,y_train)
y_val_norm<-norm(y_val,y_train)
y_test_norm<-norm(y_test,y_train)

In [599]:
y_test_norm[1:5,]

,response,response.1,response.2
118403,0.6036853,0.5568676,0.4806422
92666,0.6406169,0.7058998,0.7467189
65216,0.5619569,0.4436591,0.4224803
84384,0.7388923,0.5129699,0.7289530
45378,0.3224972,0.4131426,0.2542345


In [600]:
y_val_norm[1:5,]

,response,response.1,response.2
115139,0.4208987,0.4213394,0.4376924
124073,0.3329612,0.4139647,0.3457655
391,0.5814604,0.6142003,0.4809413
118582,0.5601739,0.4667882,0.6768027
103396,0.6301235,0.7124208,0.7679458


In [42]:
#Make phi
phi_transform<-function(X){
    phi<-cbind(bias=rep(1,nrow(X)),X)
    return(phi)
}

In [43]:
#Clean out unused variables again. Probably not necessary, but the data set is reasonably large
rm(X_train)
rm(y_train)
rm(X_val)
rm(y_val)
rm(X_test)
rm(y_test)

In [44]:
X_tr_norm<-as.matrix(X_tr_norm)
X_val_norm<-as.matrix(X_val_norm)
X_test_norm<-as.matrix(X_test_norm)
y_tr_norm<-as.matrix(y_tr_norm)
y_val_norm<-as.matrix(y_val_norm)
y_test_norm<-as.matrix(y_test_norm)

In [45]:
#Create phis for test train validate
phi_tr<-phi_transform(X_tr_norm)
phi_val<-phi_transform(X_val_norm)
phi_test<-phi_transform(X_test_norm)

In [309]:
errbest<-10000000

In [310]:
w<-matrix(rnorm(ncol(y_tr_norm)*ncol(phi_tr)),ncol(phi_tr),ncol(y_tr_norm))
w_best<-w

In [613]:
#Set the number of epochs
epochs<-20

In [644]:
#Initialize w and set hyper parameters
error<-c()
epoch<-c()
eta<-0.0000004
lam1<-.00001
lam2<-.001
w<-w_best
for (i in 1:epochs){
    #calculate error on validation set
    y_hat_val<-phi_val%*%w
    error<-c(error,sum(diag(t(y_val_norm-y_hat_val)%*%(y_val_norm-y_hat_val))))
    epoch<-c(epoch,i)
    if (error[i]<errbest){
        errbest<-error[i]
#        print("Best Error Updated")
        w_best<-w
        }
    #Calculate new w using training set
    y_hat_tr<-phi_tr%*%w
    w<-w-eta*(t(phi_tr)%*%(y_hat_tr-y_tr_norm))-lam1*sign(w)-lam2*w
    }

In [583]:
write.csv(w_best,"Xw_best.csv")

In [26]:
w_best<-as.matrix(read.csv("Xw_best.csv", header = TRUE))[,2:4]
w_best<-apply(w_best, 2, as.numeric)

In [46]:
y_hat_test<-phi_test%*%w_best
sum(diag(t(y_test_norm-y_hat_test)%*%(y_test_norm-y_hat_test)))

[1] 50.27621

In [57]:
jpeg("Categorical_Example.jpg")
par(mfrow=c(2,1))
plot(data[,4],data[,97], xlab="predictor.3", ylab="response", col="blue", cex.axis=2,cex.lab=2)
plot(data[,96],data[,97], xlab="predictor.95", ylab="response", col="blue",cex.axis=2,cex.lab=2)
dev.off()

png 
  2

In [22]:
jpeg("Linear_response.jpg")
plot(data[,3],data[,97], xlab="predictor.3", ylab="response", col="blue", cex.axis=2,cex.lab=2)
lines(data[,3], rep(2650000, length(data[,3])), col="red", lwd=5)
dev.off()

png 
  2

In [5]:
x_sq<-data[,18]**2

In [23]:
jpeg("Quadratic_esponse.jpg")
plot(data[,18],data[,97],xlab="predictor.17",ylab="response", col="blue", cex.axis=2,cex.lab=2)
lines(data[,18],2600000+ x_sq, col="red", lwd=4)
dev.off()

png 
  2

In [56]:
jpeg("results.jpg")
par(mar=c(5,5,5,5))
plot(y_test_norm[,1],y_hat_test[,1], xlab="Actual response", ylab="Predicted response", col="blue", cex.axis=2,cex.lab=2)
lines(y_test_norm[,1],y_test_norm[,1], col="red", lwd=4)
dev.off()

png 
  2